# codigo explicado

1. **Importações**: As primeiras linhas importam as bibliotecas necessárias para o código. `numpy` é uma biblioteca para manipulação de arrays, `tensorflow` é uma biblioteca para aprendizado de máquina, `tensorflow_datasets` é uma biblioteca para carregar conjuntos de dados, e `sklearn.preprocessing` contém utilitários para pré-processamento de dados.

2. **Carregando os dados**: As linhas que começam com `ratings = tfds.load('movielens/100k-ratings', split='train')` e `movies = tfds.load('movielens/100k-movies', split='train')` carregam os dados de classificações de filmes e os dados dos filmes do conjunto de dados MovieLens.

3. **Preparando os dados**: As próximas linhas extraem os IDs dos usuários, os títulos dos filmes e as classificações dos usuários dos dados carregados e os convertem em arrays numpy. Em seguida, calcula o número de usuários e filmes únicos.

4. **Definindo a arquitetura do modelo**: A classe `TwoTowersModel` define a arquitetura do modelo de sistema de recomendação. Ela herda da classe `Model` do Keras. O método `__init__` define as camadas do modelo, que incluem duas camadas de incorporação (uma para usuários e outra para filmes), uma camada para achatar as incorporações, uma camada para concatenar as incorporações achatadas, e duas camadas densas. O método `call` define como essas camadas são conectadas.

5. **Instanciando e compilando o modelo**: As linhas `model = TwoTowersModel(50, n_users, n_movies)` e `model.compile(loss='mse', optimizer=Adam())` instanciam o modelo e o compilam. O modelo é instanciado com um tamanho de incorporação de 50, e o número de usuários e filmes únicos. É compilado com a perda de erro quadrático médio e o otimizador Adam.

6. **Treinando o modelo**: A linha `model.fit([user_ids, movie_titles], user_ratings, epochs=5, verbose=1)` treina o modelo nos dados de treinamento. Os IDs dos usuários e os títulos dos filmes são passados como entrada, e as classificações dos usuários são passadas como alvos. O modelo é treinado por 5 épocas.

7. **Codificando os IDs dos usuários e os títulos dos filmes**: As linhas que começam com `user_encoder = LabelEncoder()` e `movie_encoder = LabelEncoder()` criam dois codificadores LabelEncoder, que podem converter strings em números inteiros e vice-versa. As linhas `user_ids = user_encoder.fit_transform(user_ids)` e `movie_titles = movie_encoder.fit_transform(movie_titles)` usam esses codificadores para transformar os IDs dos usuários e os títulos dos filmes em números inteiros.

Este código agora deve funcionar corretamente. Ele primeiro carrega os dados de classificações de filmes do conjunto de dados MovieLens. Em seguida, ele prepara os dados convertendo os IDs dos usuários e os títulos dos filmes em arrays numpy. Ele então codifica esses arrays como números inteiros usando LabelEncoder. Em seguida, ele define a arquitetura do modelo, que consiste em duas “torres” (uma para usuários e outra para filmes) que são alimentadas em uma rede densa para prever a classificação do usuário para um filme. Finalmente, ele treina o modelo usando os dados de classificações de filmes.

In [ ]:
!pip install tensorflow_recommenders

In [ ]:
import numpy as np
import tensorflow as tf

# isso deve ser uma warning esse traço amarelo abaixo da importação
from tensorflow.keras.layers import Embedding, Flatten, Dense, Concatenate
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam


import tensorflow_datasets as tfds
from sklearn.preprocessing import LabelEncoder

In [ ]:
# Carregar dados de classificações de filmes
ratings = tfds.load('movielens/100k-ratings', split='train')

# Preparar os dados
user_ids = np.array([x['user_id'].numpy() for x in ratings])
movie_titles = np.array([x['movie_title'].numpy().decode('utf-8') for x in ratings])
user_ratings = np.array([x['user_rating'].numpy() for x in ratings])

# Codificar os IDs dos usuários e os títulos dos filmes como números inteiros
user_encoder = LabelEncoder()
movie_encoder = LabelEncoder()

user_ids = user_encoder.fit_transform(user_ids)
movie_titles = movie_encoder.fit_transform(movie_titles)

n_users = len(user_encoder.classes_)
n_movies = len(movie_encoder.classes_)

# Definindo a arquitetura do modelo
class TwoTowersModel(Model):
    def __init__(self, embedding_size, n_users, n_movies):
        super(TwoTowersModel, self).__init__()
        self.user_embedding = Embedding(output_dim=embedding_size, input_dim=n_users, input_length=1, name='user_embedding')
        self.movie_embedding = Embedding(output_dim=embedding_size, input_dim=n_movies, input_length=1, name='movie_embedding')
        self.flatten = Flatten()
        self.concat = Concatenate()
        self.dense1 = Dense(128, activation='relu')
        self.dense2 = Dense(1)

    def call(self, inputs):
        user_inputs = inputs[0]
        movie_inputs = inputs[1]
        user_vecs = self.flatten(self.user_embedding(user_inputs))
        movie_vecs = self.flatten(self.movie_embedding(movie_inputs))
        y = self.concat([user_vecs, movie_vecs])
        y = self.dense1(y)
        y = self.dense2(y)
        return y

# Instanciando e compilando o modelo
model = TwoTowersModel(50, n_users, n_movies)
model.compile(loss='mse', optimizer=Adam())

# Treinando o modelo
model.fit([user_ids, movie_titles], user_ratings, epochs=5, verbose=1)
